In [135]:
import pandas as pd

#### Задача
* Проверьте данные на нормальность и на наличие корреляции. 
  * Для этого объедините 2 массива в DataFrame. 
  * Обоснуйте выбор теста на корреляцию. 
  * Сделайте вывод по гипотезе.
* Также ответьте на вопрос:
  * Какова разница в среднем размере мидии в зависимости от города-производителя. 
  * Обоснуйте выбор теста. 
  * Сделайте вывод по гипотезе.

### Формирование DataFrame из 2х массивов данных

In [136]:
#Наша дата-сет - это два списка данных о размерах мидий в СПб и Магадане. Скопируем его из исходника.
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

#Создадим единый dataframe с наблюденими в фомате Город, Диаметр
# Дата фрейм зададим списками по строкам Город-Наблюдение

for i in range(len(petersburg)):
    petersburg[i] = ["Спб",petersburg[i]]
for i in range(len(magadan)):
    magadan[i] = ["Магадан",magadan[i]]

#Объединим данные в единый вложенный список 
petersburg.extend(magadan) 

#Создадим наш датафрейм
df = pd.DataFrame(
    data = petersburg,
    columns= ['Город', 'Диаметр']
) 

df

,Город,Диаметр
0,Спб,0.0974
1,Спб,0.1352
2,Спб,0.0817
3,Спб,0.1016
4,Спб,0.0968
5,Спб,0.1064
6,Спб,0.1050
7,Магадан,0.1033
8,Магадан,0.0915
9,Магадан,0.0781


### Проверки на нормальность

In [137]:
#Данные из разных городов (разные наборы данных). Поэтому будем считать их независимыми. 
#На нормальность выборки я проверю и в совокупности и отдельно. Просто, чтобы разобраться, что происходит.
# Как я понял из рукододства тест shapiro все данные поданные на вход считает единой выборкой
from scipy.stats import shapiro
alpha = 0.05 #зададим точность
H0 = 'Данные распределены нормально (не можем отвергнуть гипотезу о нормальном распределении)'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

#тестируем данные в совокупности тестом Шапиро-Уилка
_, p = shapiro(df['Диаметр'])
print('p=%.3f' % p)

if p > alpha:
	print(H0, 'в совокупности')
else:
	print(Ha, 'в совокупности')
 
#тестируем данные Питера тестом Шапиро-Уилка
_, p = shapiro(df[ df['Город'] == 'Спб' ]['Диаметр'])
print('p=%.3f' % p)

if p > alpha:
	print(H0, 'по СПб')
else:
	print(Ha, 'по СПб')
 
 #тестируем данные Магадана тестом Шапиро-Уилка
_, p = shapiro(df[ df['Город'] == 'Магадан' ]['Диаметр'])
print('p=%.3f' % p)

if p > alpha:
	print(H0, 'по Магадану')
else:
	print(Ha, 'по Магадану')


p=0.115
Данные распределены нормально (не можем отвергнуть гипотезу о нормальном распределении) в совокупности
p=0.242
Данные распределены нормально (не можем отвергнуть гипотезу о нормальном распределении) по СПб
p=0.036
Данные не распределены нормально (мы отвергаем H0) по Магадану


Данные меня удивили. Получается, что выборка в совокупности скорее нормальная (правда с достаточно низким p-value), данные по СПб нормальны, а вот по Магадану получается, что H0 опровергнута.

Я даже решил посмотреть распределение на графике (ниже)

Вообще, я думаю, что такие непонятки связаны с малым количеством наблюдений в СПб и Магадане отдельно. В реальных практических задачах дефицита наблюдений быть не должно по идее...

In [138]:
# Данные меня удивили. Получается, что выборка в совокупности скорее нормальная (правда с достаточно низким p-value), 
# данные по СПб нормальны, а вот по Магадану получается, что H0 опровергнута
import plotly.express as px

fig = px.histogram(df, x='Диаметр', title='Распеделение диаметра', text_auto=True, nbins=20, color = 'Город')
fig.show()

На всякий случай, до окончательных выводов по нормальности сделаем еще и тесты Д'Агостино:
Там все также, но вот какой итог:
1. По выборке в совокупности я получил сообщение, что данные валидны от 20 наблюдений, но получил результат.
2. Маленькие же выборки Магадана и Питера тест даже не принял, с ошибкой, что значение слишком мало

Итог:
***Будем проводить исследование, как сработают разные методы***

Начнем с подхода, как к нормальной выборке, а потом попробуем непараметрические тесты.

Но в жизни, я бы сказал, что ***данных мало***, а ***на нормальность их надо проверять отдельно, а не в совокупности*** т.к. это независимые разные выборки, хоть и измерение одинаковое.

*Проверяющему ментору: просьба указать в обратной связи, согласны ли вы с последним утверждением, и, если нет, то почему*

### Определение корреляции

Тут конечно у меня большой вопрос. Что такое корреляция? Это мера зависимости в парах параметров между друг с другом.
Что в нашей задаче является наблюдением? Выбор города и измерение диаметра конкретной Мидии из этого города.

Соответсвенно корреляцию мы можем измерять только между величинами "Город" и "Диаметр".

Неверно было бы ставить вопрос о том коррелируют ли диаметры мидий из двух городов. Но это просте неверная потсановка вопроса: про разные города мы можем сказать, что это разные наблюдения только.

Переведем параметр Город в Числовой (1 = Питер, 0 = Магадан) и составим матрицу корреляций.
 Т.к. вопрос о нормальности у нас полуоткрыт на этих данных, я сделаю и тест Писона и тест Спирмана. Я уверен, что они покажут похожие результаты, а проверка не сложная.

In [139]:
df['id_города'] = df['Город'].apply(lambda x: 1 if x == 'Спб' else 0)
print('Матрица корреляции города и диаметра по Пирсону')
display(df.corr())
print('Матрица корреляции города и диаметра по Спирману')
display(df.corr(method='spearman'))

# и ради интереса посчитаем спирмана через scipy.stats
from scipy.stats import spearmanr
from scipy.stats import pearsonr
print(spearmanr(df['id_города'], df['Диаметр']))
print(pearsonr(df['id_города'], df['Диаметр']))



Матрица корреляции города и диаметра по Пирсону


,Диаметр,id_города
Диаметр,1.000000,0.683781
id_города,0.683781,1.000000


Матрица корреляции города и диаметра по Спирману


,Диаметр,id_города
Диаметр,1.000000,0.711378
id_города,0.711378,1.000000


SpearmanrResult(correlation=0.711378010251503, pvalue=0.00294008557136221)
PearsonRResult(statistic=0.6837806545855616, pvalue=0.004940324868384323)


У нас есть условный порог 0.7, чтобы считать сильную корреляцию и среднюю. Наши исследования показали результат около этого числа с разных сторон. Но по факту мы можем сказать, что обоими методами корреляция около этого порога. А значит есть существенная и даже скорее сильная корреляция города и размера мидий. Причем она положительная. А Питер у нас с номеров 1, т.е. больше номера Магадана.

Вывод:
***Мидии в Питере в среднем больше мидий из Магадана***

*не такой уж и великий вывод - все было видно на графике распределений сразу*

### Определение разницы в среднем размере мидии в зависимости от города-производителя

Ну первое что приходит в голову - вычислить эту разницу, как разницу средних значений выборок, а потом проверить гипотезу, что это то самое значение.
Опять же общая выборка ведет себя как нормальная, но наблюдений мало, а тест на нормальность "полупройден".

Поэтому делаем двумя подходами, что уж. Интересно будет понаблюдать разницу.

In [140]:
delta = df[df['Город'] == 'Спб']['Диаметр'].mean() - df[df['Город'] == 'Магадан']['Диаметр'].mean() 
delta = round(delta,5)
print(f'Разница средних диаметров равна: {delta}')

Разница средних диаметров равна: 0.02543


Теперь сделаем так: 

добавим ко всем значениям Магадана нашу вероятную дельту (по сути получим то же распределение в выбоке, только со смещением на константу) и проверим этим два распределения на совпадение средних значений.

Т.к. магадан распределен ненормально, а смещение на нормальность распределения не влияет, я бы делал тест уилкоксона (который сумма рангов) - буду искать его в документации

Но для интереса, я решил провести тест на нормальность смещенных данных в совокупности, и если будет "да", то сделать через независимый t-тест (независимый, т.к. выборки из разных городов)

In [141]:
#Создадим поле Диам Центр - оцентрированный диаметр
df['Диам Центр'] = df['Диаметр']*df['id_города'] + (df['Диаметр']+delta)*(1-df['id_города']) 
# для Спб (id =1) оставляем все как есть, а для Магадана (id = 0) смежаем на нашу дельту

#Теперь сделаем тест на нормальность новой выборки в совокупности
_, p = shapiro(df['Диам Центр'])
print('p=%.3f' % p)

if p > alpha:
	print(H0, 'в совокупности')
else:
	print(Ha, 'в совокупности')

p=0.066
Данные распределены нормально (не можем отвергнуть гипотезу о нормальном распределении) в совокупности


Тест показал, что мы не можем отмести гипотезу о нормальности. Однако p-value конечно получилось очень маленькое - при другом выборе альфа (небольшом варьировании, например до 10%) уже мог быть другой результат. Тем интереснее будет провести оба варианта тестирования.

Но по мне так, опять у нас звоночек, что нудно данные дособрать и их мало для реальной практической задачи.
А в этой ситуации предпочел бы непараметрический тест Уилкоксона.

Но начнем мы с т-теста, т.к. он был в примере и его не нужно искать в библиотеках:

In [142]:
# начнем с независимого t-теста
H0 = 'Нет разницы между скорректированной выборкой Магадана и выборкой Питера в среднем диаметре мидии'
Ha = 'Есть значимая между скорректированной выборкой Магадана и выборкой Питера в среднем диаметре мидии'

from scipy.stats import ttest_ind

print('\n' + "*** Результаты независимого T-теста ***")
test_results = ttest_ind(df[df['Город'] == 'Магадан']['Диам Центр'], df[df['Город'] == 'Спб']['Диам Центр'], equal_var=True)
p = test_results[1]
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")



*** Результаты независимого T-теста ***
0.9999628603811013 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет разницы между скорректированной выборкой Магадана и выборкой Питера в среднем диаметре мидии


p кстати получили очень близкий к 1.
Вывод: 

***можно считать, что среднии мидии отличаются на 0.02543 в зависимости от города (в Спб крупнее)***

На самом деле с учетом наших задаваемых параметров alpha, даже не смотря на мой скепсис к маленьким выборкам и p-value на нормальность близким к 7% в смещенной выборке, задача прошла чисто и можно это брать за достоверный ответ.

Но вспомним что данные Магадана отдельно вряд ли нормальны и попробуем другой подход.

In [143]:
from scipy.stats import ranksums # нам нужен вариант именно про сумму рангов т.к. выборки разные
w = ranksums(df[df['Город'] == 'Магадан']['Диам Центр'],df[df['Город'] == 'Спб']['Диам Центр'] , nan_policy='propagate')
p = w[1]

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.6434288435636205 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет разницы между скорректированной выборкой Магадана и выборкой Питера в среднем диаметре мидии


Вывод:
Непараметрический вариант теста тоже подтвердил гипотезу что delta = 0.02543 c высокой точностью.

А я нашел, как в библиотеке scipy.stats отличаются **Знаковый ранговый критерий Уилкоксона** и **критерий суммы рангов Уилкоксона**

А именно: *wiranksums* и *wilcoxon*